# Assignment Hyperparameter Optimization

In [ ]:
# !pip uninstall tf-keras
# !pip install keras-tuner
# !pip install tensorflow==2.16.1

In [ ]:
import keras
import tensorflow as tf
print("Keras Current Version:", keras.__version__, "Tensorflow Current Version:", tf.__version__)

Keras Current Version: 3.3.3 Tensorflow Current Version: 2.16.1


# Imports

In [ ]:
import numpy as np
import pandas as pd
from joblib import dump, load
import random
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import SGD, RMSprop, Adam
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.initializers import RandomNormal, RandomUniform, GlorotUniform, GlorotNormal, HeNormal
from keras.optimizers.schedules import ExponentialDecay
from keras_tuner import RandomSearch, GridSearch, BayesianOptimization
from keras_tuner.engine.hyperparameters import HyperParameters

random.seed(46)
np.random.seed(46)
tf.random.set_seed(46)

# import os
import time


# Functions

In [ ]:
def preprocess_data(filepath):
    data = pd.read_csv(filepath)
    scaler = StandardScaler()
    X = scaler.fit_transform(data.drop('Outcome', axis=1))
    y = data['Outcome'].values
    dump(scaler, 'scaler.joblib')
    return X, y

def prepare_datasets(X_train, X_val, y_train, y_val, batch_size=None):
    if batch_size is None:
        batch_size = len(X_train)
    train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
    train_dataset = train_dataset.shuffle(buffer_size=len(X_train)).batch(batch_size)
    val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
    val_dataset = val_dataset.batch(batch_size)
    return train_dataset, val_dataset

def plot_training_history(history, train_loss='loss', train_metric='accuracy', val_loss='val_loss', val_metric='val_accuracy'):

    #Loss
    plt.figure(figsize=(10, 5))
    plt.plot(history.history[train_loss], label='Training Loss')
    plt.plot(history.history[val_loss], label='Validation Loss')
    plt.title('Training and Validation Loss Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

    # Metrics
    plt.figure(figsize=(10, 5))
    plt.plot(history.history[train_metric], label=f"Training: {train_metric}")
    plt.plot(history.history[val_metric], label=f"Validation: {val_metric}")
    plt.title(f'Training and Validation {train_metric} Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel(f'train_metric')
    plt.legend()
    plt.show()

def get_best_epoch_details(history):
    val_losses = history.history['val_loss']
    min_val_loss_index = val_losses.index(min(val_losses))
    best_epoch = min_val_loss_index + 1

    epoch_details = {}
    for key in history.history.keys():
        epoch_details[key] = history.history[key][min_val_loss_index]

    epoch_details['best_epoch'] = best_epoch
    print(f"Best epoch details: {epoch_details}")

# Data Preparation

In [ ]:
X, y = preprocess_data('/content/diabetes.csv')

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

train_ds, val_ds = prepare_datasets(X_train, X_val, y_train, y_val, batch_size=32)

# Task 1: Hiperparametre arama uzayını aşağıdaki değerlere göre oluşturunuz:

**Layer sayısı**:  1-10

**Unit sayısı**: 32-512 arasında 16'şar artacak şekilde.

**Aktivasyon fonksiyonları**: relu, tanh, sigmoid

**l2**: 0.0001-0.01

**dropout**: 0.1-0.5 arasında 0.05 artacak şekilde.

**initial learning rate**: 0.0001-0.01 (1e-4 - 1e-2)

**learning rate scheduler**: decay steps: 20

**optimizer'lar**: 'sgd', 'adam', 'rmsprop'

**Random search:** epoch sayısı en az 200 olmalı

Diğer ayarları dilediğiniz gibi yapabilirsiniz.

In [ ]:
def build_model(hp):
    model = Sequential()
    model.add(Input(shape=(train_ds.element_spec[0].shape[1],)))

    # Hidden layers, activation functions, l2, Dropout
    for i in range(hp.Int('num_layers', 1, 10)):

        model.add(Dense(units=hp.Int('units_' + str(i), min_value=32, max_value=512, step=16),
                        activation=hp.Choice('activation_' + str(i), values=['relu', 'tanh', 'sigmoid']),
                        kernel_regularizer=l2(hp.Float('l2_' + str(i), min_value=0.0001, max_value=0.01, sampling='log'))))

        model.add(BatchNormalization())
        model.add(Dropout(hp.Float('dropout_' + str(i), min_value=0.0, max_value=0.5, step=0.05)))

    model.add(Dense(1, activation='sigmoid'))

    # Learning rate schedule
    initial_learning_rate = hp.Float('initial_learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')

    lr_schedule = ExponentialDecay(
        initial_learning_rate=initial_learning_rate,
        decay_steps=20,
        decay_rate=0.96,
        staircase=True
    )

    # optimizers
    optimizer_choice = hp.Choice('optimizer', values=['sgd', 'adam', "rmsprop"])
    if optimizer_choice == 'sgd':
        optimizer = SGD(
            learning_rate=lr_schedule,
            momentum=hp.Float('momentum', min_value=0.0, max_value=0.9, step=0.1)
        )
    elif optimizer_choice == 'adam':
        optimizer = Adam(
            learning_rate=lr_schedule,
            beta_1=hp.Float('beta1', min_value=0.85, max_value=0.99, step=0.01),
            beta_2=hp.Float('beta2', min_value=0.999, max_value=0.9999, step=0.0001),
            epsilon=hp.Float('epsilon', min_value=1e-8, max_value=1e-7, step=1e-8)
        )

    elif optimizer_choice == 'rmsprop':
        optimizer = RMSprop(
            learning_rate=lr_schedule,
            rho=hp.Float('rho', min_value=0.8, max_value=0.99, step=0.01),
            epsilon=hp.Float('epsilon', min_value=1e-10, max_value=1e-8, step=1e-10),
            momentum=hp.Float('momentum', min_value=0.0, max_value=0.9, step=0.1)
        )

    model.compile(optimizer=optimizer,
                  loss="binary_crossentropy",
                  metrics=["accuracy"])

    return model

# Task 2: Epoch sayısı 200 olacak şekilde aramayı başlatınız. Diğer ayarlar aynı kalabilir.



In [ ]:
random_search_tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=20,
    executions_per_trial=1,
    overwrite=True)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=20,
    verbose=1,
    restore_best_weights=True)


In [ ]:
random_search_tuner.search(train_ds,
                           epochs=200,
                           validation_data=val_ds,
                           callbacks=[early_stopping])


Trial 20 Complete [00h 00m 35s]
val_loss: 0.505858302116394

Best val_loss So Far: 0.505858302116394
Total elapsed time: 00h 10m 03s


In [ ]:
random_search_tuner.search_space_summary()

Search space summary
Default search space size: 44
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 10, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 16, 'sampling': 'linear'}
activation_0 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh', 'sigmoid'], 'ordered': False}
l2_0 (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}
dropout_0 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.05, 'sampling': 'linear'}
initial_learning_rate (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}
optimizer (Choice)
{'default': 'sgd', 'conditions': [], 'values': ['sgd', 'adam', 'rmsprop'], 'ordered': False}
momentum (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.9, 'ste

# Task 3: En iyi 3 hiperparametre setini getiriniz, ayrı ayrı kaydediniz, değerlerini inceleyiniz, bazı hiperparametre değerlerini yorumlayınız.

In [ ]:
best_hps = random_search_tuner.get_best_hyperparameters(num_trials=3)

In [ ]:
best_hps_1 = best_hps[0]
best_hps_2 = best_hps[1]
best_hps_3 = best_hps[2]

In [ ]:
print(f"Best hyperparameters: {best_hps_1.values}")

Best hyperparameters: {'num_layers': 5, 'units_0': 320, 'activation_0': 'tanh', 'l2_0': 0.000278798421624668, 'dropout_0': 0.1, 'initial_learning_rate': 0.0009018154359590027, 'optimizer': 'rmsprop', 'momentum': 0.8, 'units_1': 304, 'activation_1': 'sigmoid', 'l2_1': 0.004975241205355714, 'dropout_1': 0.35000000000000003, 'units_2': 512, 'activation_2': 'tanh', 'l2_2': 0.0030328742763881247, 'dropout_2': 0.35000000000000003, 'units_3': 416, 'activation_3': 'sigmoid', 'l2_3': 0.0052240320209991295, 'dropout_3': 0.4, 'units_4': 512, 'activation_4': 'sigmoid', 'l2_4': 0.007657649016426419, 'dropout_4': 0.35000000000000003, 'beta1': 0.97, 'beta2': 0.9991, 'epsilon': 4e-08, 'units_5': 256, 'activation_5': 'relu', 'l2_5': 0.00018823046175344533, 'dropout_5': 0.15000000000000002, 'units_6': 144, 'activation_6': 'tanh', 'l2_6': 0.005587978564753659, 'dropout_6': 0.0, 'rho': 0.9600000000000001, 'units_7': 464, 'activation_7': 'sigmoid', 'l2_7': 0.0001739367668022079, 'dropout_7': 0.25, 'units_8

In [ ]:
print(f"Best hyperparameters: {best_hps_2.values}")

Best hyperparameters: {'num_layers': 1, 'units_0': 336, 'activation_0': 'tanh', 'l2_0': 0.0014897265192904891, 'dropout_0': 0.05, 'initial_learning_rate': 0.0001661447098557725, 'optimizer': 'rmsprop', 'momentum': 0.30000000000000004, 'units_1': 144, 'activation_1': 'relu', 'l2_1': 0.005817174658255382, 'dropout_1': 0.35000000000000003, 'units_2': 400, 'activation_2': 'relu', 'l2_2': 0.006935794679708803, 'dropout_2': 0.45, 'units_3': 432, 'activation_3': 'sigmoid', 'l2_3': 0.0001522298574808984, 'dropout_3': 0.4, 'units_4': 144, 'activation_4': 'sigmoid', 'l2_4': 0.0001321561859789267, 'dropout_4': 0.30000000000000004, 'beta1': 0.89, 'beta2': 0.9996, 'epsilon': 8e-08, 'units_5': 400, 'activation_5': 'tanh', 'l2_5': 0.00031613913359087616, 'dropout_5': 0.25, 'units_6': 224, 'activation_6': 'relu', 'l2_6': 0.0011563133814945622, 'dropout_6': 0.45, 'rho': 0.8}


In [ ]:
print(f"Best hyperparameters: {best_hps_3.values}")

Best hyperparameters: {'num_layers': 5, 'units_0': 32, 'activation_0': 'sigmoid', 'l2_0': 0.002456151967584859, 'dropout_0': 0.25, 'initial_learning_rate': 0.0025415877460743122, 'optimizer': 'rmsprop', 'momentum': 0.7000000000000001, 'units_1': 176, 'activation_1': 'relu', 'l2_1': 0.00035276138234744287, 'dropout_1': 0.1, 'units_2': 384, 'activation_2': 'tanh', 'l2_2': 0.0003299839763081992, 'dropout_2': 0.1, 'units_3': 48, 'activation_3': 'tanh', 'l2_3': 0.008401740215485797, 'dropout_3': 0.35000000000000003, 'units_4': 336, 'activation_4': 'sigmoid', 'l2_4': 0.0019473554687093966, 'dropout_4': 0.15000000000000002, 'beta1': 0.97, 'beta2': 0.9998, 'epsilon': 6e-08, 'units_5': 160, 'activation_5': 'sigmoid', 'l2_5': 0.002140192612726292, 'dropout_5': 0.25, 'units_6': 96, 'activation_6': 'relu', 'l2_6': 0.00014325529176189037, 'dropout_6': 0.15000000000000002, 'rho': 0.9400000000000001, 'units_7': 352, 'activation_7': 'relu', 'l2_7': 0.0010800922509442582, 'dropout_7': 0.1, 'units_8': 6

# Task 4: En iyi 3 modeli seçiniz.

In [ ]:
best_models = random_search_tuner.get_best_models(num_models=3)

# Task 5: En iyi 3 modelin bir döngü aracılığı ile model başarısını hesaplayınız

In [ ]:
for i, model in enumerate(best_models):
    loss, acc = model.evaluate(val_ds, verbose=0)
    print(f"Model {i+1}, Validation loss: {loss}, Validation Accuracy: {acc}")

Model 1, Validation loss: 0.505858302116394, Validation Accuracy: 0.7792207598686218
Model 2, Validation loss: 0.520683228969574, Validation Accuracy: 0.7792207598686218
Model 3, Validation loss: 0.5239221453666687, Validation Accuracy: 0.7922077775001526


# Task 6: Modellerin accuracy değerleri arasında neden fark var? En tepedeki modelin en iyi olmasını bekleriz, eğer öyle değilse neden en tepedekinin accuracy değeri en yüksek değil?

In [ ]:
# Model 1, Validation loss: 0.505858302116394, Validation Accuracy: 0.7792207598686218
# Model 2, Validation loss: 0.520683228969574, Validation Accuracy: 0.7792207598686218
# Model 3, Validation loss: 0.5239221453666687, Validation Accuracy: 0.7922077775001526